In [9]:
import math
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime
from chinese_calendar import is_workday, is_holiday
import pickle as pk
from Utils import *
import Metrics

TASK = "out"

#数据采用统计周期
TIME_INTERVAL = 5
HIS_DAY = 1

#数据开始时间，1日到25日，共25天数据
DATA_START_DAY = 1
DATA_END_DAY = 25

#每天被截取的时间，早晨06:00到晚上23:30
DAYSTART = "06:00"
DAYEND = "23:30"

N_NODE = 81
#输入时间步，输出时间步
INPUT_STEP = 12
PRED_STEP = 12
HORIZON = 1

# train test 划分
TRAINRATIO = 0.8 # TRAIN + VAL
TRAINVALSPLIT = 0.125 # val_ratio = 0.8 * 0.125 = 0.1


CHANNEL = 1
BATCHSIZE = 16
LEARN = 0.001
EPOCH = 400
PATIENCE = 20

OPTIMIZER = 'RMSprop'
LOSS = 'MSE'


# read data
daystartt = datetime.datetime.strptime(DAYSTART, '%H:%M') 
dayendt = datetime.datetime.strptime(DAYEND, '%H:%M') 
day_minutes = int((dayendt - daystartt).total_seconds()/60)  #每天有多少分钟的数据
day_total_step = math.ceil(day_minutes/TIME_INTERVAL)   #对应每天有多少个step
data_in,data_out = read_file(TIME_INTERVAL,DATA_START_DAY,DATA_END_DAY,DAYSTART,DAYEND)  #data数据读取，并且截取 1号到 25号， 06:00 到 23:30的数据
if TASK is "in":
    data = data_in
else :
    data = data_out
# sequential
data = data.values.reshape(data.shape[0],data.shape[1],1)
seq_data = datasetToSeq(data, INPUT_STEP,PRED_STEP)
seq_data = seq_data[:,INPUT_STEP:INPUT_STEP+PRED_STEP,:,:]

#testYS
start_index = int(seq_data.shape[0]*TRAINRATIO)
end_index = seq_data.shape[0]
testYS = seq_data[start_index:end_index]

#historicalaverage YS_pred
YS_pred = []
for i in range(start_index,end_index):
    his_index =[j for j in range(i-210*HIS_DAY,i,210)]
    temp = seq_data[his_index,:,:,:]
    temp = np.mean(temp,axis=0)
    YS_pred.append(temp)
YS_pred = np.array(YS_pred)
print('YS_pred shape: ', YS_pred.shape)
MODELNAME = 'HistoricalAverage-yindu'
PATH = './save'


def testModel(name, mode, YS, YS_pred):
    print('TIMESTEP_IN, TIMESTEP_OUT', INPUT_STEP, PRED_STEP)
    print('YS.shape, YS_pred.shape,', YS.shape, YS_pred.shape)
    np.save(PATH + '/' + MODELNAME + '_prediction.npy', YS_pred)
    np.save(PATH + '/' + MODELNAME + '_groundtruth.npy', YS)
    MSE, RMSE, MAE, MAPE = Metrics.evaluate(YS, YS_pred)
    print('*' * 40)
    f = open(PATH + '/' + name + '_prediction_scores.txt', 'a')
    print("all pred steps, %s, %s, MSE, RMSE, MAE, MAPE, %.10f, %.10f, %.10f, %.10f" % (name, mode, MSE, RMSE, MAE, MAPE))
    f.write("for past %s days, all pred steps, %s, %s, MSE, RMSE, MAE, MAPE, %.10f, %.10f, %.10f, %.10f\n" % (str(HIS_DAY), name, mode, MSE, RMSE, MAE, MAPE))
    for i in range(PRED_STEP):
        MSE, RMSE, MAE, MAPE = Metrics.evaluate(YS[:, i, :, :], YS_pred[:, i, :, :])
        print("%d step, %s, %s, MSE, RMSE, MAE, MAPE, %.10f, %.10f, %.10f, %.10f" % (i+1, name, mode, MSE, RMSE, MAE, MAPE))
        f.write("%d step, %s, %s, MSE, RMSE, MAE, MAPE, %.10f, %.10f, %.10f, %.10f\n" % (i+1, name, mode, MSE, RMSE, MAE, MAPE))
    f.close()
    
testModel(MODELNAME, 'TEST', testYS, YS_pred)

YS_pred shape:  (1046, 12, 81, 1)
TIMESTEP_IN, TIMESTEP_OUT 12 12
YS.shape, YS_pred.shape, (1046, 12, 81, 1) (1046, 12, 81, 1)
****************************************
all pred steps, HistoricalAverage-yindu, TEST, MSE, RMSE, MAE, MAPE, 1661.2187817199, 40.7580517410, 19.7404869816, 45.5813190508
1 step, HistoricalAverage-yindu, TEST, MSE, RMSE, MAE, MAPE, 1660.6581804877, 40.7511739768, 19.7187758185, 45.7608596998
2 step, HistoricalAverage-yindu, TEST, MSE, RMSE, MAE, MAPE, 1660.5800934778, 40.7502158703, 19.7195784057, 45.7418495156
3 step, HistoricalAverage-yindu, TEST, MSE, RMSE, MAE, MAPE, 1660.6054693955, 40.7505272284, 19.7213724240, 45.7020914947
4 step, HistoricalAverage-yindu, TEST, MSE, RMSE, MAE, MAPE, 1660.7729504520, 40.7525821323, 19.7262587635, 45.6569005258
5 step, HistoricalAverage-yindu, TEST, MSE, RMSE, MAE, MAPE, 1660.9035361046, 40.7541842773, 19.7305549654, 45.6559383665
6 step, HistoricalAverage-yindu, TEST, MSE, RMSE, MAE, MAPE, 1661.0404126242, 40.7558635367,